<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a1e01a8e01360e038ea70e6fadc5c0d64ab20f4639ce2349dff899c56a1fbd3e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 14:10:44
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -83.41 K (-0.58%)
Current PnL: -17.54 L (-11.8%)
CY Booked + Current PnL: -6.10 L (-4.1%)
-------------------
Total profit:  1.91 L
Total loss:  -19.44 L
-------------------
Total Booked + Current PnL: 20.88 L (17.16%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 83.05 L (58.26%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199480.0,34810.0,8518.0,1.58,21.14,4.27,26.31,89.0,4.09,1.43,30.81,X40N,NTT,AC
76,TTKPRESTIG,770.00,99.79,46.0,M-SC,3.33,86067.0,-14710.0,14804.0,-1.10,-14.60,17.20,0.09,245.0,-0.99,0.62,13.07,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.59,112434.0,11930.0,17180.0,0.26,11.87,15.28,28.96,79.0,0.69,0.81,49.87,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,219055.0,8689.0,20087.0,-0.23,4.13,9.17,13.68,37.0,0.43,1.57,21.34,XY25,NTT,REFINERIES
41,ITC,452.00,-38.34,45.0,H-LC,2.28,199047.0,-1091.0,21975.0,-0.88,-0.55,11.04,10.44,4.0,-0.05,1.43,4.99,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1254.07,80.0,M-SC,6.40,184130.0,2987.0,42479.0,10.45,1.65,23.07,25.10,235.0,0.07,1.32,47.56,XY24,NTT,HEALTHCARE
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142382.0,-40393.0,151466.0,2.07,-22.10,106.38,60.77,125.0,-0.27,1.02,27.53,XR,NTT,JEWELLERY
31,HINDZINC,730.22,22.76,63.0,M-LC,1.68,200535.0,-4541.0,117112.0,1.75,-2.21,58.40,54.89,52.0,-0.04,1.44,21.35,X5K,ATH,METALS
34,ICICIPRULI,790.00,-19.62,53.0,H-MC,2.17,138402.0,2614.0,40137.0,1.63,1.93,29.00,31.48,107.0,0.07,0.99,14.28,X40,ATH,INSURANCE
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199480.0,34810.0,8518.0,1.58,21.14,4.27,26.31,89.0,4.09,1.43,30.81,X40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.0,1793.94,53.0,L-SC,2.44,51680.0,-85497.0,85587.0,-3.71,-62.33,165.61,0.07,267.0,-1.00,0.37,28.53,OX40N,NTT,JEWELLERY
83,VOLTAS,1530.0,-1.78,46.0,H-MC,2.74,206040.0,14298.0,23468.0,-3.26,7.46,11.39,19.69,99.0,0.61,1.48,14.76,XY25,NTT,AC
81,VALIANTORG,838.0,-319.59,48.0,H-SC,6.01,122060.0,-41545.0,162865.0,-3.13,-25.39,133.43,74.15,139.0,-0.26,0.88,56.46,XR,NTT,CHEMICALS
39,INFY,2275.0,-18.16,48.0,H-LC,8.20,318691.0,5803.0,165879.0,-2.86,1.85,52.05,54.87,3.0,0.03,2.29,8.63,X40,BTT,IT
66,SONACOMS,1006.0,-40.39,39.0,M-SC,11.72,74191.0,-26968.0,101857.0,-2.81,-26.66,137.29,74.03,202.0,-0.26,0.53,4.62,AR,BTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.93,53.0,H-SC,16.80,200415.0,461.0,96760.0,-0.9,0.23,48.28,48.63,115.0,0.00,1.44,11.81,XR,NTT,MISC
29,HAVELLS,2069.16,0.20,58.0,H-MC,1.93,248414.0,683.0,74375.0,0.1,0.28,29.94,30.30,92.0,0.01,1.78,13.60,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,99.79,46.0,M-SC,3.33,86067.0,-14710.0,14804.0,-1.10,-14.60,17.20,0.09,245.0,-0.99,0.62,13.07,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.30,121554.0,-20925.0,40441.0,-1.68,-14.69,33.27,13.70,163.0,-0.52,0.87,43.47,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,62.0,H-SC,0.82,226809.0,-42858.0,79202.0,0.06,-15.89,34.92,13.47,138.0,-0.54,1.63,14.72,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,46.0,H-SC,2.14,145544.0,-30359.0,72379.0,1.02,-17.26,49.73,23.89,149.0,-0.42,1.04,25.70,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.19,47.0,H-MC,7.26,106590.0,-25245.0,67471.0,-1.97,-19.15,63.30,32.03,98.0,-0.37,0.76,19.39,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.6,69.0,H-MC,3.52,199480.0,34810.0,8518.0,1.58,21.14,4.27,26.31,89.0,4.09,1.43,30.81,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1254.07,80.0,M-SC,6.40,184130.0,2987.0,42479.0,10.45,1.65,23.07,25.10,235.0,0.07,1.32,47.56,XY24,NTT,HEALTHCARE
59,SAIL,228.00,49.16,71.0,M-MC,8.23,235570.0,10608.0,157055.0,0.44,4.72,66.67,74.53,192.0,0.07,1.69,38.49,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199480.0,34810.0,8518.0,1.58,21.14,4.27,26.31,89.0,4.09,1.43,30.81,X40N,NTT,AC
29,HAVELLS,2069.16,0.20,58.0,H-MC,1.93,248414.0,683.0,74375.0,0.10,0.28,29.94,30.30,92.0,0.01,1.78,13.60,X40,ATH,ELECTRICAL
17,CAMS,5211.76,-1.17,59.0,H-SC,3.10,112364.0,10360.0,33563.0,-2.14,10.16,29.87,43.06,122.0,0.31,0.81,29.01,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.59,112434.0,11930.0,17180.0,0.26,11.87,15.28,28.96,79.0,0.69,0.81,49.87,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.04,49.0,M-SC,0.82,99720.0,8222.0,70532.0,-0.61,8.99,70.73,86.07,223.0,0.12,0.72,44.81,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,58.0,M-LC,5.66,158646.0,7701.0,101756.0,-0.41,5.10,64.14,72.51,53.0,0.08,1.14,11.43,XR,NTT,IT
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.61,185790.0,5982.0,106421.0,0.27,3.33,57.28,62.52,88.0,0.06,1.33,38.55,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.33,33.0,H-SC,11.36,130375.0,7039.0,126724.0,-0.39,5.71,97.20,108.45,119.0,0.06,0.93,29.34,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.48,38.0,H-LC,6.86,296989.0,-18853.0,131507.0,-1.89,-5.97,44.28,35.67,5.0,-0.14,2.13,7.06,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.83,226394.0,-15530.0,78808.0,-1.73,-6.42,34.81,26.15,8.0,-0.20,1.62,6.98,X40,ATH,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.28,199047.0,-1091.0,21975.0,-0.88,-0.55,11.04,10.44,4.0,-0.05,1.43,4.99,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-13.32,46.0,H-LC,7.73,218733.0,-33035.0,86246.0,0.16,-13.12,39.43,21.13,27.0,-0.38,1.57,17.66,X40,ATH,PAINTS
39,INFY,2275.00,-18.16,48.0,H-LC,8.20,318691.0,5803.0,165879.0,-2.86,1.85,52.05,54.87,3.0,0.03,2.29,8.63,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.00,69.0,H-LC,1.47,247928.0,1777.0,77428.0,-0.27,0.72,31.23,32.18,16.0,0.02,1.78,37.02,X200,ATH,IT
41,ITC,452.0,-38.34,45.0,H-LC,2.28,199047.0,-1091.0,21975.0,-0.88,-0.55,11.04,10.44,4.0,-0.05,1.43,4.99,X40,NTT,FMCG
64,SIEMENS,4671.5,1.67,58.0,H-LC,3.57,163550.0,-22545.0,70032.0,-1.65,-12.11,42.82,25.51,15.0,-0.32,1.17,20.52,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.75,50.0,H-LC,4.61,277805.0,12379.0,44421.0,-0.61,4.66,15.99,21.40,11.0,0.28,1.99,12.81,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,54.0,H-LC,4.82,219055.0,8689.0,20087.0,-0.23,4.13,9.17,13.68,37.0,0.43,1.57,21.34,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-38.34,45.0,H-LC,2.28,199047.0,-1091.0,21975.0,-0.88,-0.55,11.04,10.44,4.0,-0.05,1.43,4.99,X40,NTT,FMCG
79,UNITDSPR,1644.00,-11.95,56.0,H-LC,7.66,230772.0,-3372.0,51993.0,1.02,-1.44,22.53,20.77,86.0,-0.06,1.65,5.38,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.14,5.53,X40,ATH,IT
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.83,226394.0,-15530.0,78808.0,-1.73,-6.42,34.81,26.15,8.0,-0.20,1.62,6.98,X40,ATH,IT
82,VBL,671.64,-17.48,38.0,H-LC,6.86,296989.0,-18853.0,131507.0,-1.89,-5.97,44.28,35.67,5.0,-0.14,2.13,7.06,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,1.67,58.0,H-LC,3.57,163550.0,-22545.0,70032.0,-1.65,-12.11,42.82,25.51,15.0,-0.32,1.17,20.52,AR,ATH,ELECTRICAL
41,ITC,452.00,-38.34,45.0,H-LC,2.28,199047.0,-1091.0,21975.0,-0.88,-0.55,11.04,10.44,4.0,-0.05,1.43,4.99,X40,NTT,FMCG
19,CIPLA,1795.00,-20.47,48.0,H-LC,5.52,212624.0,8124.0,33297.0,-1.51,3.97,15.66,20.25,10.0,0.24,1.52,13.05,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.32,46.0,H-LC,7.73,218733.0,-33035.0,86246.0,0.16,-13.12,39.43,21.13,27.0,-0.38,1.57,17.66,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,219055.0,8689.0,20087.0,-0.23,4.13,9.17,13.68,37.0,0.43,1.57,21.34,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.14,5.53,X40,ATH,IT
39,INFY,2275.00,-18.16,48.0,H-LC,8.20,318691.0,5803.0,165879.0,-2.86,1.85,52.05,54.87,3.0,0.03,2.29,8.63,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.28,199047.0,-1091.0,21975.0,-0.88,-0.55,11.04,10.44,4.0,-0.05,1.43,4.99,X40,NTT,FMCG
82,VBL,671.64,-17.48,38.0,H-LC,6.86,296989.0,-18853.0,131507.0,-1.89,-5.97,44.28,35.67,5.0,-0.14,2.13,7.06,X40N,ATH,FMCG
1,ABB,7934.00,-37.73,66.0,H-LC,9.40,260352.0,-1267.0,120491.0,-0.24,-0.48,46.28,45.57,7.0,-0.01,1.87,10.78,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6977.78,62.0,L-SC,5.14,77663.0,-16147.0,92768.0,-0.83,-17.21,119.45,81.67,269.0,-0.17,0.56,50.40,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.29,82810.0,-30739.0,70794.0,-2.68,-27.07,85.49,35.27,268.0,-0.43,0.59,102.68,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,66.0,M-SC,3.92,84924.0,-17286.0,110673.0,-1.17,-16.91,130.32,91.37,208.0,-0.16,0.61,37.86,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.51,44.0,H-SC,12.30,87974.0,-12989.0,107522.0,-1.54,-12.87,122.22,93.63,148.0,-0.12,0.63,31.03,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94515.0,-3465.0,25065.0,-0.10,-3.54,26.52,22.05,152.0,-0.14,0.68,37.38,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1254.07,80.0,M-SC,6.40,184130.0,2987.0,42479.0,10.45,1.65,23.07,25.10,235.0,0.07,1.32,47.56,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.40,73.0,H-SC,13.44,226578.0,-44183.0,374126.0,-1.76,-16.32,165.12,121.86,133.0,-0.12,1.62,77.25,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,66.0,M-SC,3.92,84924.0,-17286.0,110673.0,-1.17,-16.91,130.32,91.37,208.0,-0.16,0.61,37.86,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142382.0,-40393.0,151466.0,2.07,-22.10,106.38,60.77,125.0,-0.27,1.02,27.53,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3743.24,57.0,M-SC,4.96,119841.0,-27527.0,79347.0,-1.23,-18.68,66.21,35.16,236.0,-0.35,0.86,22.41,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.58
1,25,44.21
2,50,75.33


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.82
LC    31.28
MC    23.87
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.85
X40      15.01
X40N     11.61
XR       11.48
AR        8.55
XY25      8.38
OX40N     7.95
X200      1.78
MH        1.68
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.90
H-LC    24.68
H-MC    20.89
M-SC    13.40
M-LC     5.55
M-MC     2.68
L-SC     1.52
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.29,-13.41,75.24
FMCG,12.27,-4.86,42.37
FINANCE,10.22,-15.17,65.41
MISC,7.19,-14.55,78.23
BANKS,6.18,-15.62,76.44
PAINTS,5.74,-15.53,32.75
ELECTRICAL,5.62,-5.09,41.97
INSURANCE,3.75,-5.65,45.15
AC,3.62,4.00,12.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3080038.0,22
XR,1309566.0,14
AR,1136984.0,9
X40,755820.0,10
X40N,585537.0,9
OX40N,536630.0,10
XY25,378470.0,6
SR,248050.0,2
X5K,117112.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3622597.0,29
M-SC,1377811.0,17
H-MC,1191522.0,15
H-LC,1085589.0,14
M-LC,376848.0,4
M-MC,303563.0,2
L-SC,249149.0,3
L-MC,60484.0,1
L-LC,37729.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1214003.0      6
M-SC       XY24       788526.0      7
H-SC       AR         782235.0      5
           XR         763459.0      7
H-MC       XY24       555457.0      4
H-LC       X40        475824.0      5
M-MC       XY24       303563.0      2
H-SC       X40N       296929.0      4
           OX40N      255444.0      4
           SR         248050.0      2
H-LC       X40N       216797.0      3
H-MC       X40        207622.0      4
H-LC       AR         190523.0      2
H-MC       XY25       165560.0      2
M-SC       AR         164226.0      2
L-SC       XR         163562.0      2
M-LC       XY24       157980.0      2
M-SC       OX40N      128128.0      4
M-LC       X5K        117112.0      1
M-SC       XR         113884.0      2
           XY25       110673.0      1
H-MC       XR         106421.0      1
M-LC       XR         101756.0      1
L-SC       OX40N       85587.0      1
H-LC       X200        77428.0      1
M-SC       X40         72374.0      1
H-MC       X40N        71811.0      2
           OX40N       67471.0      1
H-LC       XY25        64508.0      2
H-SC       MH          62477.0      1
H-LC       XY24        60509.0      1
L-MC       XR          60484.0      1
L-LC       XY25        37729.0      1
H-MC       MH          17180.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
